In [2]:
import pandas as pd
import numpy as np
from typing import Dict, List
import math
from datetime import datetime, timezone
from scipy.stats import skew, kurtosis

In [3]:
def parse_timestamp_to_utc(timestamp):
    """밀리초 단위 유닉스 타임스탬프를 UTC datetime으로 변환"""
    try:
        # 밀리초를 초로 변환
        ts_seconds = float(timestamp) / 1000
        return datetime.fromtimestamp(ts_seconds, tz=timezone.utc)
    except (ValueError, TypeError):
        raise ValueError(f"Invalid timestamp format: {timestamp}")

In [4]:
data = pd.read_csv(r"C:\Dev\TMBearing\testdata.csv")  

# UTC 시간 및 초 단위 컬럼 추가
data['time'] = data['timestamp'].apply(parse_timestamp_to_utc)
data['second'] = (data['timestamp'].apply(float) / 1000).astype(int)  # 초 단위로 내림

In [5]:
data

,magic,deviceID,commandId,timestamp,ch1,ch2,ch3,time,second
0,aaaa,2732,10,1734381207178,-0.001391,-0.002563,-1.025016,2024-12-16 20:33:27.178000+00:00,1734381207
1,aaaa,2732,10,1734381207179,-0.001406,-0.002703,-1.025359,2024-12-16 20:33:27.179000+00:00,1734381207
2,aaaa,2732,10,1734381207180,-0.001172,-0.003031,-1.025375,2024-12-16 20:33:27.180000+00:00,1734381207
3,aaaa,2732,10,1734381207181,-0.001156,-0.003172,-1.025375,2024-12-16 20:33:27.181000+00:00,1734381207
4,aaaa,2732,10,1734381207182,-0.001203,-0.003016,-1.026000,2024-12-16 20:33:27.182000+00:00,1734381207
...,...,...,...,...,...,...,...,...,...
44578375,aaaa,2732,10,1734427292363,-0.000750,-0.002406,-1.027016,2024-12-17 09:21:32.363000+00:00,1734427292
44578376,aaaa,2732,10,1734427292364,-0.000797,-0.002422,-1.025516,2024-12-17 09:21:32.364000+00:00,1734427292
44578377,aaaa,2732,10,1734427292365,-0.000094,-0.002797,-1.025938,2024-12-17 09:21:32.365000+00:00,1734427292
44578378,aaaa,2732,10,1734427292366,-0.000266,-0.002688,-1.026109,2024-12-17 09:21:32.366000+00:00,1734427292


In [ ]:
# Time feature 생성
def rms(x): return np.sqrt(np.mean(np.square(x)))
def ptp(x): return np.ptp(x)
def sra(x): return np.square(np.mean(np.sqrt(np.abs(x))))
def aa(x):  return np.mean(np.abs(x))
def var(x): return np.var(x)
def skewness(x): return skew(x, bias=False)
def kurt(x): return kurtosis(x, bias=False)
def crest_factor(x): return np.max(np.abs(x)) / rms(x)
def shape_factor(x): return rms(x) / aa(x)
def impulse_factor(x): return np.max(np.abs(x)) / aa(x)
def coefficient_of_variation(x): return np.max(np.abs(x)) / sra(x)
def coefficient_of_skewness(x): return skewness(x) / var(x)**3
def coefficient_of_kurtosis(x): return kurt(x) / var(x)**4

In [ ]:
statistics_functions = {
    'mean': 'mean',
    'rms': rms,
    'sra': sra,
    'aa': aa,
    'max': 'max',
    'min': 'min',
    'ptp': ptp,
    'var': var,
    'skewness': skewness,
    'kurtosis': kurt,
    'crest': crest_factor,
    'shape': shape_factor,
    'impulse': impulse_factor,
    'coefficient_of_variation': coefficient_of_variation,
    'coefficient_of_skewness': coefficient_of_skewness,
    'coefficient_of_kurtosis': coefficient_of_kurtosis
}

In [12]:
# 메인 함수: 채널별 딕셔너리 형태로 반환
def compute_channel_stats_by_dict(df, channels=['ch1', 'ch2', 'ch3'], group_col='second'):
    grouped = df.groupby(group_col)
    results = {}

    for ch in channels:
        stats_df = grouped[ch].agg(list(statistics_functions.values())).reset_index()
        stats_df.columns = [group_col] + list(statistics_functions.keys())
        results[ch] = stats_df

    return results

In [13]:
results = compute_channel_stats_by_dict(data)
print(results['ch1'].head())

       second      mean       rms       sra        aa       max       min  \
0  1734381207 -0.001232  0.001264  0.001215  0.001232 -0.000344 -0.002203   
1  1734381208 -0.001230  0.001265  0.001211  0.001230 -0.000109 -0.002125   
2  1734381209 -0.001243  0.001274  0.001226  0.001243 -0.000359 -0.002156   
3  1734381210 -0.001221  0.001254  0.001203  0.001221 -0.000078 -0.002062   
4  1734381211 -0.001226  0.001343  0.001208  0.001251  0.001969 -0.007703   

        ptp           var  skewness   kurtosis     crest     shape  
0  0.001859  8.083356e-08 -0.049185   0.105329  1.742813  1.026294  
1  0.002016  8.701787e-08  0.044461  -0.121791  1.680467  1.028373  
2  0.001797  7.774912e-08  0.030462   0.043285  1.692892  1.024859  
3  0.001984  8.096267e-08  0.053202  -0.031183  1.644770  1.026783  
4  0.009672  3.021399e-07 -1.414794  26.446340  5.734487  1.073806  
